
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0)                                |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer                       |

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
!pip install boto3

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 131 kB 34.5 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 13.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.7 MB 76.5 MB/s eta 0:00:01
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
     |████████████████████████████████| 140 kB 90.2 MB/s eta 0:00:01
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, ArrayType
from pyspark.sql.functions import col, monotonically_increasing_id
import pyspark.sql.functions as f
import boto3
from pyspark.sql.window import Window
w = Window.partitionBy(f.lit(1)).orderBy(f.lit(1))


In [ ]:
schema_fact_song = StructType([
                    StructField('song_id', StringType(), True),
                    StructField('artist_id', StringType(), True),
                    StructField('release', StringType(), True),
                    StructField('title', StringType(), True),
                    StructField('duration', DoubleType(), True),
                    StructField('loudness', DoubleType(), True),
                    StructField('artist_hotttnesss', DoubleType(), True),
                    StructField('year', IntegerType(), True),
                    StructField('start_of_fade_out', DoubleType(), True),
                    StructField('end_of_fade_in', DoubleType(), True)
])

schema_dim_location = StructType([
                    StructField('artist_location', StringType(), True)                 
])

schema_dim_album = StructType([
                    StructField('release', StringType(), True)
])

schema_dim_artist = StructType([
                    StructField('artist_id', StringType(), True),
                    StructField('artist_location', StringType(), True),
                    StructField('artist_familiarity', DoubleType(), True),
                    StructField('artist_hotttnesss', DoubleType(), True),
                    StructField('artist_name', StringType(), True),
                    StructField('artist_longitude', DoubleType(), True),
                    StructField('artist_latitude', DoubleType(), True)
])

schema_dim_term = StructType([
                    # StructField('term_id', IntegerType(), True),
                    StructField('artist_terms', StringType(), True)
])

schema_dim_artist_term = StructType([
                    StructField('artist_terms', StringType(), True),
                    StructField('artist_id', StringType(), True)
])


In [ ]:
PARQUET_PATH = 's3://1msongdata/final_parquet/C/A.parquet'

In [ ]:
def parquet_to_table(parquet_path):
    dataFrame = spark.read.parquet(parquet_path)
    numOfDataPoint = dataFrame.count()

    # create dim_term table
    dim_term_table = dataFrame.filter((f.size(dataFrame.artist_terms) != 0)).select(f.explode(dataFrame.artist_terms).alias('terms'))
    dim_term_table = dim_term_table.distinct().withColumn('term_id', f.row_number().over(w))
    newRow_term = spark.createDataFrame([('unknown',-1)])
    dim_term_table = dim_term_table.union(newRow_term).sort('term_id')
    
    # create dim_artist_term table
    dim_artist_term_table = dataFrame.select('artist_id', 'artist_terms')
    dim_artist_term_table = dim_artist_term_table.withColumn('artist_terms', \
        f.when(f.size(f.col('artist_terms')) == 0, f.array(f.lit('unknown')))\
        .otherwise(f.col('artist_terms')))

    dim_artist_term_table = dim_artist_term_table.select('artist_id', f.explode(dim_artist_term_table.artist_terms).alias('terms')).distinct()
    dim_artist_term_table = dim_artist_term_table.join(dim_term_table, 'terms').select('artist_id', 'term_id')

    # create dim_album table
    col_dim_album = dataFrame.select(
        col('release'),
    )
    dim_album_table = spark.createDataFrame(col_dim_album.collect(), schema_dim_album)
    dim_album_table = dim_album_table.select("*").withColumn("album_id", monotonically_increasing_id())
    dim_album_table = dim_album_table.na.fill(value='unknown')

    
    # create dim_location table
    col_dim_location = dataFrame.select(
        col('artist_location')
    )
    dim_location_table = spark.createDataFrame(col_dim_location.collect(), schema_dim_location)
    dim_location_table = dim_location_table.select("*").withColumn("location_id", monotonically_increasing_id())
    dim_location_table = dim_location_table.na.fill(value='unknown')
    
    
    # create fact_song table
    col_fact_song = dataFrame.select(
        col("song_id"),
        col("artist_id"),
        col('release'),
        col("title"),
        col("duration"),
        col("loudness"),
        col("artist_hotttnesss"),
        col("year"),
        col("start_of_fade_out"),
        col("end_of_fade_in")
    )
    fact_song_table = spark.createDataFrame(col_fact_song.collect(), schema_fact_song)
    fact_song_table = dim_album_table.join(fact_song_table, "release").select(col("song_id"),col("artist_id"),col('release'),col("title"),col("duration"),col("loudness"),col("artist_hotttnesss"),col("year"),col("start_of_fade_out"),col("end_of_fade_in"),col("album_id"))
    fact_song_table = fact_song_table.na.fill(value='unknown')

    
    # create dim_artist table
    col_dim_artist = dataFrame.select(
        col('artist_id'),
        col('artist_location'),
        col('artist_familiarity'),
        col('artist_hotttnesss'),
        col('artist_name'),
        col('artist_longitude'),
        col('artist_latitude')
    )
    dim_artist_table = spark.createDataFrame(col_dim_artist.collect(), schema_dim_artist)
    dim_artist_table= dim_artist_table.join(dim_location_table, "artist_location").select(col('artist_id'), col('artist_location'), col('artist_familiarity'), col('artist_hotttnesss'), col('artist_name'), col('artist_longitude'), col('artist_latitude'), col("location_id"))
    dim_artist_table = dim_artist_table.na.fill(value='unknown')
    
    return dim_term_table, dim_artist_term_table, dim_album_table, dim_location_table, fact_song_table, dim_artist_table


In [ ]:
# Apply parquet_to_table Function

dim_term_table, dim_artist_term_table, dim_album_table, dim_location_table, fact_song_table, dim_artist_table = parquet_to_table(PARQUET_PATH)

In [ ]:
# Write parquet file to S3

dim_term_table.write.parquet("s3://1msongdata/Clean_parquet/data_for_test/dim_term_table.parquet")
dim_artist_term_table.write.parquet("s3://1msongdata/Clean_parquet/data_for_test/dim_artist_term_table.parquet")
dim_album_table.write.parquet("s3://1msongdata/Clean_parquet/data_for_test/dim_album_table.parquet")
dim_location_table.write.parquet("s3://1msongdata/Clean_parquet/data_for_test/dim_location_table.parquet")
dim_artist_table.write.parquet("s3://1msongdata/Clean_parquet/data_for_test/dim_artist_table.parquet")
fact_song_table.write.parquet("s3://1msongdata/Clean_parquet/data_for_test/fact_song_table.parquet")